# Collision-Free Multi-Vehicle Routing

## Functions 

In [137]:
def get_unique_nodes(edge_list):
    """
    Creates a list of all unique nodes appearing in a given graph.
    The start node is added by default; every other node is only included
    if it can be reached from another node. Isolated nodes will not be 
    added to the list, unless they happen to be the start node.
    """
    
    nr_nodes = 0
    unique_nodes = []
    unique_nodes.append(start_node)

    for edge in edge_list:
        if edge[1] not in unique_nodes:
            unique_nodes.append(edge[1])

    return unique_nodes


In [138]:
def construct_sliced_graph(edge_list, start, dest):
    """
    Constructs a two-dimensional array from a list of edges. Each subarray represents 
    one time slice. The returned array contains the specified start node in the 
    first sub-array; based on this start node, every node that can be reached in 
    n steps is present in the nth subarray. Nodes that can be reached in a different 
    amount of steps are present in every fitting subarray.
    """

    sg = [] # sliced graph
    sg.append([start])
    #node_counter = 1


    nr_unique_nodes = len(get_unique_nodes(edge_list))

    for k in range(nr_unique_nodes):
    
        if len(sg[k]) != 0 and not (len(sg[k]) == 1 and sg[k][0] == dest):
            sg.append([])
            for j in range(len(edge_list)):
                # for a given edge j in the format a->b:
                # if a is in previous slice and b is not in this slice, and if b is not the start node, add it to the slice
                if edge_list[j][0] in sg[k] and edge_list[j][1] not in sg[k+1] and edge_list[j][1] != start:
                    sg[k+1].append(edge_list[j][1])
        else:
            break


    # necessary if graph is not acyclic
    if dest in sg[-1]:
        sg[-1] = [dest]
    else:
        return None 
        # TODO - error handling

    # the last node is always the destination node,
    # therefore we can eliminate all nodes in the next-to-last slice that do not lead to the destination node.
    # going back that way, we can eliminate nodes that won't lead to our destination.
    nr_slices_to_remove = 0

    for i in range(len(sg) - 1):
        nodes_to_remove = []
        index = len(sg) - 1 - i
        for j in range(len(sg[index - 1])):
            leads_to_next_slice = 0 # number of edges in the next slice that sg[index-1][j] is connected to
            for k in range(len(sg[index])):

                # for each node in the next slice, add 1 if sg[index-1][j] is connected to it, 0 if not
                leads_to_next_slice += next((1 for u, v in enumerate(edge_list) if v[0] == sg[index-1][j] and v[1] == sg[index][k]), 0) 

            # if leads_to_next_slice is zero, it means sg[index-1][j] is not connected to any node in the next slice; essentially a dead end. 
            # it is removed, and any node in the previous slice that only connected to it will thus also be a dead end, and will be removed. etc., etc.
            if leads_to_next_slice == 0:
                nodes_to_remove.append(sg[index-1][j])

        for node in nodes_to_remove:
            sg[index-1].remove(node)

    slices_to_remove = []
    for i in range(len(sg)):
        if len(sg[len(sg) - 1 - i]) == 1 and dest in sg[len(sg) - 1 - i]:
            slices_to_remove.append(len(sg) - 1 - i)
        else:
            break
    
    if len(slices_to_remove) > 1:
        for i in slices_to_remove[:-1]:
            del sg[i]
            # print('deleted slice ' + str(i))

    return sg

In [139]:
def construct_numbered_sliced_graph(sg, offset):
    """
    Based on a two-dimensional array that contains time slices from a graph (sliced graph),
    renumbers the contents by giving each element its index in the corresponding 
    flattened array. Since the numbered sliced graph is used for getting the qubit 
    indices of the nodes, and since there are several sliced graphs, an offset is
    necessary.
    Example: With input ([[0], [1, 3], [2, 3]], 0), output would be [[0], [1, 2], [3, 4]].
    With input ([[0], [1, 3], [2, 3], [4, 3]], 5), output would be [[5], [6, 7], [8, 9], [10, 11]].
    """
    
    nsg = [] # numbered sliced graph
    node_counter = 0

    for m in range(len(sg)):
        nsg.append([])
        for n in range(len(sg[m])):
            nsg[m].append(node_counter + offset)
            node_counter+=1
    
    return nsg, node_counter

In [140]:
def construct_graph_matrix(edge_list):
    """
    Constructs a dataframe that contains all edges present in 
    the graph. The returned dataframe can contain several edges 
    per two nodes. The entry at matrix[0][1] contains an array 
    of all edges that go from node 0 to node 1.
    """

    node_labels = get_unique_nodes(edge_list)
    graph_matrix = pd.DataFrame([[[] for _ in range(len(node_labels))] for _ in range(len(node_labels))], node_labels, node_labels)
    
    for edge in edge_list:
        start = edge[0]
        end = edge[1]
        graph_matrix[start][end].append(edge)
    
    for i in range(nr_vehicles): # this is because here we use the graph's edges for all the vehicles, but we added the 0-weighted edges to the subgraphs before. they need to be added here as well. admittedly not elegant; might be changed later. also - check for duplicate dest nodes, and don't add the same 0-weight edge twice # TODO
        dest = vehicles[i][1]
        graph_matrix[dest][dest].append((dest, dest, 0.0))
    
    return graph_matrix

In [141]:
def construct_cost_matrix(edge_matrix):
    """
    Constructs a dataframe that contains the cost for the edge 
    between any given pair of nodes a and b. If there is no 
    outgoing edge from a to b, the cost will be equal to the
    fixed penalty value. If there is exactly one edge, its cost
    will be used. If there are several, the smallest cost is 
    used.
    """

    node_labels = edge_matrix.columns
    cost_matrix = pd.DataFrame([[[] for _ in range(len(node_labels))] for _ in range(len(node_labels))], node_labels, node_labels)

    for i in node_labels:
        for j in node_labels:
            entries = edge_matrix[i][j]
            if len(entries) == 0:
                cost_matrix[i][j] = (penalty_value)
            elif len(entries) == 1:
                cost_matrix[i][j] = (entries[0][2])
            else:
                best_cost = penalty_value # penalty by default bigger than the biggest edge cost
                for entry in entries:
                    if entry[2] < best_cost:
                        best_cost = entry[2]
                cost_matrix[i][j] = best_cost
    return cost_matrix

In [142]:
def D_function(node, time_slice, nr_vehicles):
    """
    This function computes 'the sum over all binary variables 
    associated to [a given node] which appear in the the same time 
    slice of multiple vehicles'.
    """

    vehicles_at_position = 0
    for i in range(nr_vehicles):
        if len(test_sg[i]) > time_slice and node in test_sg[i][time_slice]:
            index = find_index(node, time_slice, test_sg[i], test_nsg[i])
            vehicles_at_position += X[index]

    return vehicles_at_position

In [143]:
def find_index(node, time_slice, sliced_graph, numbered_sliced_graph):
    """
    For a given node, time slice, and sliced graph, determines the qubit index 
    of the node. If the node is not present in the given slice in the graph, 
    returns -1.
    """
    
    for i in range(len(sliced_graph[time_slice])):
        if sliced_graph[time_slice][i] == node:
            return numbered_sliced_graph[time_slice][i]
    else:
        return -1


In [144]:
def fix_nodes(start, destination, vehicle_index, sliced_graph, numbered_sg, cost_func):
    """
    In the given cost function, fixes specific nodes according to already known 
    values for the qubits. The qubits representing the start and destination nodes 
    are fixed at 1; any other nodes in the last time slice are fixed at 0.
    """

    # set start node to 1
    start_node_index = 'X_' + str(find_index(start, 0, sliced_graph, numbered_sg))
    cost_func.linear_constraint(linear={start_node_index: 1}, sense='==', rhs=1, name=('start_node' + '_veh_' + str(vehicle_index)))

    # set destination node to 1
    dest_node_index = 'X_' + str(find_index(destination, -1, sliced_graph, numbered_sg))
    cost_func.linear_constraint(linear={dest_node_index: 1}, sense='==', rhs=1, name='destination_node' + '_veh_' + str(vehicle_index))

    # no need to set other nodes in the first and last slice to 0 because there are none (see preprocessing).

    return

In [145]:
def get_node_info_from_index(node_index, sg, nsg):
    """ For a given qubit index, returns the corresponding time slice and the node label. """

    for time_slice in range(len(nsg)):
        if node_index in nsg[time_slice]:

            pos = nsg[time_slice].index(node_index)
            node_label = sg[time_slice][pos]

            return time_slice, node_label

In [175]:
def result_evaluation(result, offset, sg, nsg, cost_matrix):
    """
    For a given qubit string, returns an explanation of the nodes that are passed per time slice.
    """
    
    counter = 0
    previous_node = None
    total_cost = 0
    solution_not_valid = False

    for i in range(len(result)):
        if result[i] == '1':
            time_slice, node_label = get_node_info_from_index(i + offset, sg, nsg)
            if time_slice != counter:
                solution_not_valid = True

            if counter == 0:
                previous_node = node_label
            else:
                cost = cost_matrix[previous_node][node_label]
                total_cost += cost
                print(('\t\t\t\tcost: {}').format(cost))
                previous_node = node_label


            print(('node {} at time slice {}').format(node_label, time_slice))

            counter += 1

            
    print(('\ntotal cost: {}').format(total_cost))
    if total_cost >= penalty_value or solution_not_valid:
        print('\nIt seems like this solution is not valid; it disregards at least one constraint.')

    return

## Preprocessing

In [147]:
import sympy as sym
import matplotlib.pyplot as plt
import pandas as pd

In [148]:
# there are two examples defined in the following, each has a graph as a collection of edges with the format (start, end, edge weight)
# as well as a defined start node and an array of vehicles [start, destination]
# actually, it's kind of redundant to have the start node in there twice but you could set it up so that there is no predefined start node and instead,
# only the vehicles' start nodes are taken into account. anyway, choose one of the examples below and comment the other out;
# if you keep both uncommented the second one is executed.

# the penalty value can be changed

# --- first example ---
graph = [(0, 1, 6.0), (0, 2, 4.0), (0, 3, 1.0), (0, 5, 2.0), 
(1, 6, 1.0), 
(2, 4, 3.0), (2, 6, 2.0), (2, 9, 5.0), 
(3, 2, 2.0), (3, 4, 4.0), 
(4, 7, 4.0), (4, 9, 2.0), 
(5, 1, 5.0), 
(6, 8, 3.0), (6, 9, 3.0), 
(7, 9, 3.0),
(9, 8, 4.0)]

start_node = 0
vehicles = [[0, 4], [0, 6]]

# --- second example ---

graph = [(0, 1, 5.0), (0, 2, 4.0), (0, 3, 4.0),
(1, 4, 3.0),
(2, 1, 2.0), (2, 3, 1.0),
(3, 4, 2.0), (3, 5, 3.0),
(4, 6, 2.0), 
(5, 7, 4.0)
]

start_node = 0
vehicles = [[0, 4], [0, 5]]

In [149]:
penalty_value = 50

nr_vehicles = len(vehicles)
subgraphs = []
nodes_in_subgraphs = []
sliced_subgraphs = []

In [150]:
for i in range(nr_vehicles):
    subgraph = [edge for edge in graph if edge[0] != vehicles[i][1]]
    subgraph.append((vehicles[i][1], vehicles[i][1], 0.0))
    subgraphs.append(subgraph)

In [151]:
unique_nodes = get_unique_nodes(graph)

for i in range(nr_vehicles):
    nodes_in_subgraphs.append(get_unique_nodes(subgraphs[i]))

In [152]:
test_sg = []
test_nsg = []

nodes_total = 0
for i in range(nr_vehicles):
    test_sg.append(construct_sliced_graph(subgraphs[i], vehicles[i][0], vehicles[i][1]))

    numbered_subgraph, nr_nodes_subgraph = construct_numbered_sliced_graph(test_sg[i], offset=nodes_total)
    test_nsg.append(numbered_subgraph)

    nodes_total += nr_nodes_subgraph

nr_qubits = nodes_total
nr_qubits

14

In [153]:
test_sg

[[[0], [1, 2, 3], [4, 1, 3], [4]], [[0], [2, 3], [3, 5], [5]]]

In [154]:
test_sg

[[[0], [1, 2, 3], [4, 1, 3], [4]], [[0], [2, 3], [3, 5], [5]]]

In [155]:
test_nsg

[[[0], [1, 2, 3], [4, 5, 6], [7]], [[8], [9, 10], [11, 12], [13]]]

In [156]:
matrix = construct_graph_matrix(graph)
matrix

,0,1,2,3,4,5,6,7
0,[],[],[],[],[],[],[],[]
1,"[(0, 1, 5.0)]",[],"[(2, 1, 2.0)]",[],[],[],[],[]
2,"[(0, 2, 4.0)]",[],[],[],[],[],[],[]
3,"[(0, 3, 4.0)]",[],"[(2, 3, 1.0)]",[],[],[],[],[]
4,[],"[(1, 4, 3.0)]",[],"[(3, 4, 2.0)]","[(4, 4, 0.0)]",[],[],[]
5,[],[],[],"[(3, 5, 3.0)]",[],"[(5, 5, 0.0)]",[],[]
6,[],[],[],[],"[(4, 6, 2.0)]",[],[],[]
7,[],[],[],[],[],"[(5, 7, 4.0)]",[],[]


In [157]:
# not necessary, but allows for easier comprehension as well as changes in cost values;
# if specific nodes should be discouraged from traversing, changes can be made 
# here instead of in the graph itself
edge_cost_matrix = construct_cost_matrix(matrix)
edge_cost_matrix

,0,1,2,3,4,5,6,7
0,50,50,50,50,50,50,50,50
1,5,50,2,50,50,50,50,50
2,4,50,50,50,50,50,50,50
3,4,50,1,50,50,50,50,50
4,50,3,50,2,0,50,50,50
5,50,50,50,3,50,0,50,50
6,50,50,50,50,2,50,50,50
7,50,50,50,50,50,4,50,50


In [158]:
highest_slice_nr = 0
for i in range(nr_vehicles):
    if len(test_sg[i]) > highest_slice_nr:
        highest_slice_nr = len(test_sg[i])

## Cost Function

In [159]:
# define all necessary variables

X = sym.IndexedBase('X') # counter variable for qubits
c = sym.symbols('c') # counter variable for slices
v = sym.symbols('v') # counter variable for vertices (nodes)
y = sym.symbols('y') # counter variable for vertices
w = sym.symbols('w') # counter variable for vertices
k = sym.symbols('k') # counter variable for vehicles
P = sym.symbols('P') # penalty value
c_max = sym.symbols('c_max') # max number of slices

nrvehicles = sym.symbols('nrvehicles')
len_unique_nodes = sym.symbols('len_unique_nodes') # number of unique vertices

uniques = sym.IndexedBase('uniques') # used to access array of unique vertices

d = sym.IndexedBase('d') # represents the function d, which returns the edge cost between two vertices
D = sym.IndexedBase('D') # represents the function D_function (see above)

sg = sym.IndexedBase('sg')
nsg = sym.IndexedBase('nsg')

nrslices = sym.IndexedBase('nrslices')
lenslice = sym.IndexedBase('lenslice')

In [160]:
cost_function = sym.Sum(
                sym.Sum(                                
                    (sym.Sum(  
                        X[nsg[k,c,v]],            
                        (v, 0, lenslice[k,c] - 1)
                        )
                        - 1 )**2 * P,

                    (c, 0, nrslices[k]-1)) + 0.5 * sym.Sum(   
                        2 * (sym.Sum(  sym.Sum(  X[nsg[k,c,y]] * X[nsg[k,c+1,w]] * d[sg[k,c,y], sg[k,c+1,w]],  (w, 0, lenslice[k,c + 1] - 1) ) , (y, 0, lenslice[k,c] - 1))),
                        (c, 0, nrslices[k] - 2)), (k, 0, nrvehicles-1)) + P * sym.Sum( sym.Sum( ( D[uniques[v],c,nrvehicles] - 1) * D[uniques[v],c,nrvehicles] , (v, 1, len_unique_nodes-1))  , (c, 0, c_max - 1)) # v is one because we don't check for node 0, since they all start there

# workaround -- multiplying with 0.5 and then 2, since sympy does not evaluate the triple sum otherwise
                    
cost_function

P*Sum((D[uniques[v], c, nrvehicles] - 1)*D[uniques[v], c, nrvehicles], (v, 1, len_unique_nodes - 1), (c, 0, c_max - 1)) + Sum(Sum(P*(Sum(X[nsg[k, c, v]], (v, 0, lenslice[k, c] - 1)) - 1)**2, (c, 0, nrslices[k] - 1)) + 0.5*Sum(2*Sum(X[nsg[k, c + 1, w]]*X[nsg[k, c, y]]*d[sg[k, c, y], sg[k, c + 1, w]], (w, 0, lenslice[k, c + 1] - 1), (y, 0, lenslice[k, c] - 1)), (c, 0, nrslices[k] - 2)), (k, 0, nrvehicles - 1))

In [161]:
# translation of data into dictionaries for sympy
single_valued_dict = {
    nrvehicles: nr_vehicles,
    P: penalty_value,
    c_max: highest_slice_nr,
    len_unique_nodes: len(unique_nodes)
    }

nr_slices_dict = {
    nrslices[k]: len(test_sg[k]) for k in range(nr_vehicles)
}

unique_nodes_dict = {
    uniques[v]: unique_nodes[v] for v in range(len(unique_nodes))
}

numbered_sliced_graph_dict = {
    nsg[k, i, j]: test_nsg[k][i][j] for k in range(nr_vehicles) for i in range(len(test_nsg[k])) for j in range(len(test_nsg[k][i]))
}

D_function_dict = {
    D[v, c, r]: D_function(v, c, r) for v in range(len(unique_nodes)) for c in range(highest_slice_nr) for r in range(nr_vehicles, nr_vehicles+1)
}

sliced_graph_dict = {
    sg[k, i, j]: test_sg[k][i][j] for k in range(nr_vehicles) for i in range(len(test_sg[k])) for j in range(len(test_sg[k][i]))
}

len_slice_dict = {
    lenslice[k, i]: len(test_sg[k][i]) for k in range(nr_vehicles) for i in range(len(test_sg[k]))
}

d_dict = {
    d[i, j]: edge_cost_matrix[i][j] 
    for i in unique_nodes
    for j in unique_nodes
}

# definition of the cost polynomial
cost_poly = sym.Poly(cost_function
                     .subs(single_valued_dict)
                     .doit()
                     .subs(nr_slices_dict)
                     .doit()
                     .subs(unique_nodes_dict)
                     .doit()
                     .subs(len_slice_dict)
                     .doit()
                     .subs(D_function_dict)
                     .doit()
                     .subs(numbered_sliced_graph_dict)
                     .subs(sliced_graph_dict)
                     .doit()
                     .subs(d_dict)
                     .doit(),
                     [X[i] for i in range(nr_qubits)])
cost_poly

# the only variables in the cost polynomial should be X_0, X_1, X_2 etc. if not, then some of the variables have not been evaluated; order of evaluation is important.
# if the evaluation of parts of the polynomial depends on the evaluation of another, then doit() is necessary in between in order to evaluate that first part.

Poly(50.0*X[0]**2 + 5.0*X[0]*X[1] + 4.0*X[0]*X[2] + 4.0*X[0]*X[3] - 100.0*X[0] + 100.0*X[1]**2 + 100.0*X[1]*X[2] + 100.0*X[1]*X[3] + 3.0*X[1]*X[4] + 50.0*X[1]*X[5] + 50.0*X[1]*X[6] - 150.0*X[1] + 100.0*X[2]**2 + 100.0*X[2]*X[3] + 50.0*X[2]*X[4] + 2.0*X[2]*X[5] + 1.0*X[2]*X[6] + 100.0*X[2]*X[9] - 150.0*X[2] + 100.0*X[3]**2 + 2.0*X[3]*X[4] + 50.0*X[3]*X[5] + 50.0*X[3]*X[6] + 100.0*X[3]*X[10] - 150.0*X[3] + 100.0*X[4]**2 + 100.0*X[4]*X[5] + 100.0*X[4]*X[6] - 150.0*X[4] + 100.0*X[5]**2 + 100.0*X[5]*X[6] + 3.0*X[5]*X[7] - 150.0*X[5] + 100.0*X[6]**2 + 2.0*X[6]*X[7] + 100.0*X[6]*X[11] - 150.0*X[6] + 100.0*X[7]**2 - 150.0*X[7] + 50.0*X[8]**2 + 4.0*X[8]*X[9] + 4.0*X[8]*X[10] - 100.0*X[8] + 100.0*X[9]**2 + 100.0*X[9]*X[10] + 1.0*X[9]*X[11] + 50.0*X[9]*X[12] - 150.0*X[9] + 100.0*X[10]**2 + 50.0*X[10]*X[11] + 3.0*X[10]*X[12] - 150.0*X[10] + 100.0*X[11]**2 + 100.0*X[11]*X[12] + 3.0*X[11]*X[13] - 150.0*X[11] + 100.0*X[12]**2 - 150.0*X[12] + 100.0*X[13]**2 - 150.0*X[13] + 400.0, X[0], X[1], X[2], X[3

## Setup

In [162]:
import qiskit
from qiskit.algorithms import QAOA, VQE

from qiskit_optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer, CplexOptimizer
from qiskit.utils import QuantumInstance
from qiskit_optimization.problems import QuadraticProgram
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP


# generate qiskit's cost function
qiskit_cost_function = QuadraticProgram()

# define qiskit variables
for i in range(nr_qubits):
    qiskit_cost_function.binary_var('X_' + str(i))

# specify qiskit cost function
qiskit_cost_function.minimize(
    linear = [int(cost_poly.coeff_monomial(X[i]**1)) for i in range(nr_qubits)],
    quadratic = {
        ('X_'+str(i), 'X_'+str(j)): cost_poly.coeff_monomial(X[i]**1 * X[j]**1)
        for i in range(nr_qubits)
        for j in range(i,nr_qubits)
    }
    )
for i in range(nr_vehicles):
    fix_nodes(vehicles[i][0], vehicles[i][1], i, test_sg[i], test_nsg[i], qiskit_cost_function)

print(qiskit_cost_function.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 100 X_0 - 150 X_1 - 150 X_2 - 150 X_3 - 150 X_4 - 150 X_5 - 150 X_6
      - 150 X_7 - 100 X_8 - 150 X_9 - 150 X_10 - 150 X_11 - 150 X_12 - 150 X_13
      + [ 100 X_0^2 + 10 X_0*X_1 + 8 X_0*X_2 + 8 X_0*X_3 + 200 X_1^2
      + 200 X_1*X_2 + 200 X_1*X_3 + 6 X_1*X_4 + 100 X_1*X_5 + 100 X_1*X_6
      + 200 X_2^2 + 200 X_2*X_3 + 100 X_2*X_4 + 4 X_2*X_5 + 2 X_2*X_6
      + 200 X_2*X_9 + 200 X_3^2 + 4 X_3*X_4 + 100 X_3*X_5 + 100 X_3*X_6
      + 200 X_3*X_10 + 200 X_4^2 + 200 X_4*X_5 + 200 X_4*X_6 + 200 X_5^2
      + 200 X_5*X_6 + 6 X_5*X_7 + 200 X_6^2 + 4 X_6*X_7 + 200 X_6*X_11
      + 200 X_7^2 + 100 X_8^2 + 8 X_8*X_9 + 8 X_8*X_10 + 200 X_9^2
      + 200 X_9*X_10 + 2 X_9*X_11 + 100 X_9*X_12 + 200 X_10^2 + 100 X_10*X_11
      + 6 X_10*X_12 + 200 X_11^2 + 200 X_11*X_12 + 6 X_11*X_13 + 200 X_12^2
      + 200 X_13^2 ]/2
Subject To
 start_node_veh_0: X_0 = 1
 destination_node_veh_0: X_7 = 1
 star

In [163]:
# the offset is not included in the cost function that will be passed to the algorithm, so it will be added later to get the actual result
# (only linear and quadratic terms are included)
offset = cost_poly.coeff_monomial(1)
offset

400.000000000000

## QAOA

In [164]:
# execute QAOA on local simulator
maxiter = 200
optimizer = SPSA(maxiter=maxiter) # JRL: was COBYLA
backend = qiskit.Aer.get_backend('qasm_simulator')

qaoa = QAOA(reps=10, optimizer=optimizer, quantum_instance =
             QuantumInstance(backend=backend, skip_qobj_validation=False))
optimizer_qaoa = MinimumEigenOptimizer(qaoa)

result_qaoa = optimizer_qaoa.solve(qiskit_cost_function)

results = []

for i in range(10):
    print(i)
    result_qaoa = optimizer_qaoa.solve(qiskit_cost_function)
    results.append(result_qaoa)

0
1
2
3
4
5
6
7
8
9


In [165]:
result_df = pd.DataFrame(columns = ['actual_opt_cost', 'path'])

for r in results:

    path_string = str(r.x).replace(' ', '').replace('.', '')[1:-1]
    result_df = result_df.append({'actual_opt_cost': r.fval + offset, 'path': path_string}, ignore_index=True)

print("QAOA:")
print(result_df.sort_values(by=['actual_opt_cost']))

QAOA:
    actual_opt_cost            path
6  15.0000000000000  11001001101011
4  16.0000000000000  10100101101011
7  16.0000000000000  10100101101011
3  56.0000000000000  10011001100011
5  56.0000000000000  10011001100011
0  57.0000000000000  10100011100011
1  59.0000000000000  10000011101011
2  61.0000000000000  10101001101011
8  62.0000000000000  11001001110011
9  102.000000000000  10000011100011


In [176]:
# sixth qubit result of the qaoa, shown as path
for i in range(nr_vehicles):
    section_start = test_nsg[i][0][0]
    section_end = test_nsg[i][-1][-1] + 1
    qubit_section_for_vehicle = result_df.iloc[6,1][section_start:section_end]

    result_evaluation(qubit_section_for_vehicle, section_start, test_sg[i], test_nsg[i], edge_cost_matrix)

node 0 at time slice 0
				cost: 5.0
node 1 at time slice 1
				cost: 3.0
node 4 at time slice 2
				cost: 0.0
node 4 at time slice 3

total cost: 8.0
node 0 at time slice 0
				cost: 4.0
node 3 at time slice 1
				cost: 3.0
node 5 at time slice 2
				cost: 0.0
node 5 at time slice 3

total cost: 7.0


## VQE

In [170]:
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP

backend = qiskit.Aer.get_backend('qasm_simulator')
optimizer = SPSA(maxiter=250)
ry = TwoLocal(nr_qubits, 'ry', 'cz', reps=10, entanglement='linear') # ansatz

vqe = VQE(ry, optimizer=optimizer, quantum_instance=QuantumInstance(backend=backend))

optimizer_vqe = MinimumEigenOptimizer(vqe)

results_vqe = []

for i in range(10):
    print(i)
    result_vqe = optimizer_vqe.solve(qiskit_cost_function)
    results_vqe.append(result_vqe)

0
1
2
3
4
5
6
7
8
9


In [171]:
# actual_opt_cost: optimization cost. includes path cost and incurred penalties. always > 0.
# path: qubit-result
result_df_vqe = pd.DataFrame(columns = ['actual_opt_cost', 'path'])

for r in results_vqe:

    path_string = str(r.x).replace(' ', '').replace('.', '')[1:-1]
    result_df_vqe = result_df_vqe.append({'actual_opt_cost': r.fval + offset, 'path': path_string}, ignore_index=True)

print("VQE:")
print(result_df_vqe.sort_values(by=['actual_opt_cost']))

VQE:
    actual_opt_cost            path
3  16.0000000000000  10100101101011
9  59.0000000000000  10011001100101
1  62.0000000000000  11001001110011
7  62.0000000000000  10100101100101
8  100.000000000000  10001001100011
2  103.000000000000  10001001100101
0  111.000000000000  10100001101101
4  111.000000000000  11001001100111
5  162.000000000000  10100111100001
6  209.000000000000  10010111100011


In [177]:
# print the fourth result of the vqe as a path
# each vehicle's path gets evaluated separately, so this is why we need each result's section start and end
for i in range(nr_vehicles):
    section_start = test_nsg[i][0][0]
    section_end = test_nsg[i][-1][-1] + 1

    # for printing another result, change the first number in the iloc call
    qubit_section_for_vehicle = result_df_vqe.iloc[4,1][section_start:section_end]

    result_evaluation(qubit_section_for_vehicle, section_start, test_sg[i], test_nsg[i], edge_cost_matrix)

node 0 at time slice 0
				cost: 5.0
node 1 at time slice 1
				cost: 3.0
node 4 at time slice 2
				cost: 0.0
node 4 at time slice 3

total cost: 8.0
node 0 at time slice 0
				cost: 4.0
node 3 at time slice 2
				cost: 3.0
node 5 at time slice 2
				cost: 0.0
node 5 at time slice 3

total cost: 7.0

It seems like this solution is not valid; it disregards at least one constraint.


## CplexOptimizer

In [173]:
# ------ CplexOptimizer needs to be installed for this and the next cell to work ------

#optimizer = CplexOptimizer() if CplexOptimizer.is_cplex_installed() else None
#results_classic = []

#for i in range(1):
#    result = optimizer.solve(qiskit_cost_function)
#    print(result)
#    results_classic.append(result)

In [174]:
### actual_opt_cost: optimization cost. includes path cost and incurred penalties. always > 0.
### path: qubit-result


# result_df_classic = pd.DataFrame(columns = ['actual_opt_cost', 'path'])

# for r in results_classic:
#     path_string = str(r.x).replace(' ', '').replace('.', '')[1:-1]
#     result_df_classic = result_df_classic.append({'actual_opt_cost': r.fval + offset, 'path': path_string}, ignore_index=True)

# print("CPlexOptimizer:")
# print(result_df_vqe.sort_values(by=['actual_opt_cost']))

# to do
- alle funktionen kommentieren -- done!!
- bei allen drei cases die richtigen parameter eintragen
- einmal tests laufen lassen
- keine deutschen kommentare? -- checked!!
- auf die test cases achten und für jeden case zwei graphen (und die richtigen anfangs- und endnodes) eintragen in separaten zellen - done